In [3]:
import numpy as np
import opendssdirect as dss
import matplotlib.pyplot as plt
import pandas as pd
import re
import sys
from lib.compute_H import compute_KCL_matrices
from lib.NR3_staticsplit import NR3_function

In [4]:
slackidx = 0
Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])
fn = 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
#fn = 'IEEE_13Node_Modified_01.dss'

times = np.linspace(0, 2*np.pi, 102)
#var = (1 + 0.1*np.sin(2*np.pi*0.01*t))
#3rd to last index is the time to evaluate var at
#penultimate index is der
#final index is capacitance

VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
    NR3_function(fn, slackidx, Vslack, None, None, -1, 0, 0,None, None )

Iteration number 0.000000
Iteration number 1.000000
Iteration number 2.000000
[0 1 2 3 3 5]
[1 2 3 4 5 6]
[[1 1 1 1 1 1 0]
 [1 1 1 1 0 1 1]
 [1 1 1 1 1 0 0]]
[[0. 0. 1. 1. 1. 1. 0.]
 [0. 0. 1. 1. 0. 1. 1.]
 [0. 0. 1. 1. 1. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
[[0.        +0.j         0.        +0.j         0.17320508+0.04330127j
  0.17320508+0.04330127j 0.17320508+0.04330127j 0.17320508+0.04330127j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.17320508+0.04330127j
  0.17320508+0.04330127j 0.        +0.j         0.17320508+0.04330127j
  0.17320508+0.08660254j]
 [0.        +0.j         0.        +0.j         0.17320508+0.04330127j
  0.17320508+0.04330127j 0.17320508+0.04330127j 0.        +0.j
  0.        +0.j        ]]
VNR
[[ 1.        +0.00000000e+00j  0.99999891+3.19546010e-06j
   0.99999782+6.39092019e-06j  0.99999575+1.12041374e-05j
   0.9

In [ ]:

def bus_phases(): #goes through all the buses and saves their phases to a list stored in a dictionary
    #1 if phase exists, 0 o.w.
    #list goes [a, b, c]
    #key is the bus name (without the phase part)
        dictionary = {}
        for k2 in range(len(dss.Circuit.AllNodeNames())):
            for i in range(1, 4):
                pattern = r"\.%s" % (str(i))

                m = re.findall(pattern, dss.Circuit.AllNodeNames()[k2])
                a, b = dss.Circuit.AllNodeNames()[k2].split('.')
                if m and a in dictionary:
                    temp = dictionary[a]
                    temp[i - 1] = 1
                    dictionary[a] = temp
                elif m and a not in dictionary:
                    dictionary[a] = [0, 0, 0]
                    temp = dictionary[a]
                    temp[i - 1] = 1
                    dictionary[a] = temp
        return dictionary


In [ ]:
for k1 in range(len(dss.Loads.AllNames())):
    print(dss.Loads.AllNames()[k1])
    dss.Loads.Name(dss.Loads.AllNames()[k1])
    print(dss.CktElement.BusNames())

In [ ]:
def d_factor(busname, cplx, ph): #pass in bus phase, re/im, phase
    Sbase = 1e6
    var = 1
    for n in range(len(dss.Loads.AllNames())): #go through the loads
        dss.Loads.Name(dss.Loads.AllNames()[n]) #set the load
        print(dss.CktElement.BusNames())
        if busname in dss.CktElement.BusNames()[0]: #check is the busname in the busname of the load
            print('hi')
            pattern =  r"\.%s" % (str(ph + 1)) #if it is, is the phase present?
            m = re.findall(pattern, dss.CktElement.BusNames()[0])
            if m: 
                print('hey')
                load_phases = [0, 0, 0]
                for i in range(1, 4): #if the phase is present, what other phases are
                    pattern = r"\.%s" % (str(i))
                    m = re.findall(pattern, dss.CktElement.BusNames()[0])
                    if m:
                        load_phases[i - 1] = 1
                print(sum(load_phases))
                if cplx == 0: #figure it out for multiphase
                    if sum(load_phases) == 1:
                        return dss.Loads.kW()*1*var*1e3/Sbase/sum(load_phases)
                    else:
                        return (dss.Loads.kW() + dss.Loads.kvar())*1*var*1e3/Sbase/sum(load_phases)
                else:
                    if sum(load_phases) == 1:
                        return dss.Loads.kvar()*1*var*1e3/Sbase/sum(load_phases)
                    else:
                        return (dss.Loads.kW() + dss.Loads.kvar())*1*var*1e3/Sbase/sum(load_phases)
 
    return 0
d_factor('675', 0, 2)

In [ ]:

def bus_phases(): #goes through all the buses and saves their phases to a list stored in a dictionary
    #1 if phase exists, 0 o.w.
    #list goes [a, b, c]
    #key is the bus name (without the phase part)
        dictionary = {}
        for k2 in range(len(dss.Circuit.AllNodeNames())):
            for i in range(1, 4):
                pattern = r"\.%s" % (str(i))

                m = re.findall(pattern, dss.Circuit.AllNodeNames()[k2])
                a, b = dss.Circuit.AllNodeNames()[k2].split('.')
                if m and a in dictionary:
                    temp = dictionary[a]
                    temp[i - 1] = 1
                    dictionary[a] = temp
                elif m and a not in dictionary:
                    dictionary[a] = [0, 0, 0]
                    temp = dictionary[a]
                    temp[i - 1] = 1
                    dictionary[a] = temp
        return dictionary

In [ ]:
fn = 'IEEE_13Node_Modified_01.dss'
fn = 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
dss.run_command('Redirect ' + str(fn))

def d_factor(busname, cplx, ph): #pass in bus phase, re/im, phase (zero-indexed)
    Sbase = 1e6
    var = 1
    for n in range(len(dss.Loads.AllNames())): #go through the loads
        dss.Loads.Name(dss.Loads.AllNames()[n]) #set the load
        print(dss.CktElement.BusNames())
        if busname in dss.CktElement.BusNames()[0]: #check is the busname in the busname of the load
            pattern =  r"\.%s" % (str(ph + 1)) #if it is, is the phase present?
            m = re.findall(pattern, dss.CktElement.BusNames()[0])
            if m: 
                load_phases = [0, 0, 0]
                for i in range(1, 4): #if the phase is present, what other phases are
                    pattern = r"\.%s" % (str(i))
                    m = re.findall(pattern, dss.CktElement.BusNames()[0])
                    if m:
                        load_phases[i - 1] = 1
                print(load_phases)
                if cplx == 0: #figure it out for multiphase
                    if sum(load_phases) == 1:
                        print('hey')
                        return dss.Loads.kW()*1*var*1e3/Sbase
                    else:
                        
                        return (dss.Loads.kW() + dss.Loads.kvar())*1*var*1e3/Sbase/sum(load_phases)
                else:
                    if sum(load_phases) == 1:
                        return dss.Loads.kvar()*1*var*1e3/Sbase
                    else:
                        return (dss.Loads.kW() + dss.Loads.kvar())*1*var*1e3/Sbase/sum(load_phases)
 
    return 0

d_factor('a05', 0, 0)

In [ ]:
def get_bus_idx(bus):
        k = -1
        for n in range(len(dss.Circuit.AllBusNames())): #iterates over all the buses to see which index corresponds to bus
            if dss.Circuit.AllBusNames()[n] in bus:
                k = n
        return k